## 初始化操作，加入必要引用
每次都需要第一个执行的

In [22]:
%matplotlib inline
from pymongo import MongoClient
from collections import defaultdict
import pandas as pd
import numpy as np
import jieba
import matplotlib
from matplotlib.widgets import Slider
import matplotlib.pyplot as plt
mongo_client = MongoClient()
db = mongo_client.jd
product_col = db['products']
comment_col = db['comments']
marked_col = db['markedComments']
user_col = db['users']
tags_col = db['tags']

## 将商品描述的词汇加入jieba的词典

加入jieba字典，并且计算各个词语的词频。

In [23]:
keys = defaultdict(lambda: 0)
for product in product_col.find().sort('_id', -1):
    for k in product.keys():
        keys[k] += 1
unused_words = ['commentCount', 'skuid', '材质/工艺', '_id', '功\u3000率（W）', '电\u3000压', 'scoreCount', 'productId']
for unused in unused_words:
    del keys[unused]
ready_to_add_words = list(keys.keys())
ready_to_add_words += ['材质', '工艺', '功率', '电压', '轻薄']

for w in ready_to_add_words:
    jieba.add_word(w, tag='nz')

## 标记重复数据
根据评论数据里面的用户注册时间进行聚合，然后通过编辑距离计算多个评论之间的相似程度。

In [20]:
from leven import levenshtein
from bson import ObjectId
pipelines = [
    {'$group': {'_id': "$userRegisterTime", 'count': {'$sum': 1}, 'contents': {'$push': {'id': '$_id', 'content': '$content'}}}},
    {'$match': {'count': {'$gt': 4}}}
]
count = 0
for user_comments in comment_col.aggregate(pipelines):
    last_comment = None
    for comment in user_comments['contents']:
        if len(comment['content']) > 7:
            if not last_comment:
                last_comment = comment
            else:
                distance = levenshtein(last_comment['content'], comment['content'])
                if distance < 5:
                    print(distance)
                    print(last_comment['content'], comment['content'])
                    comment_col.update_one({'_id': ObjectId(comment['id'])}, {'$set': {'duplicate': True}})
                    count += 1
            last_comment = comment

print('Total Count: ', count)

0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
很好不错有需要会再来的给公司买的 很好不错有需要会再来的给公司买的
0
东西不错，会继续关注 东西不错，会继续关注
0
东西不错，性价比高，性能稳定，外观漂亮！ 东西不错，性价比高，性能稳定，外观漂亮！
0
东西不错，性价比高，性能稳定，外观漂亮！ 东西不错，性价比高，性能稳定，外观漂亮！
0
东西不错，性价比高，性能稳定，外观漂亮！ 东西不错，性价比高，性能稳定，外观漂亮！
0
东西不错，性价比高，性能稳定，外观漂亮！ 东西不错，性价比高，性能稳定，外观漂亮！
0
公司使用，产品还算不错，评价点个赞。 公司使用，产品还算不错，评价点个赞。
0
外观很漂亮 性价比高 外观很漂亮 性价比高
0
喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。 喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。
0
喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。 喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。
0
喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。 喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。
0
更多优惠及技术支持联系直线0411-82373589，刘相凤工程师协助订购！ 更多优惠及技术支持联系直线0411-82373589，刘相凤工程师协助订购！
0
产品不错，非常满意。 产品不错，非常满意。
0
产品不错，非常满意。 产品不错，非常满意。
0
产品不错，非常满意。 产品不错，非常满意。
0
产品不错，非常满意。 产品不错，非常满意。
0
产品不错，非常满意。 产品不

## 生成稀疏矩阵

对评论使用TD-IDF提取关键词。并构造一个稀疏矩阵。

| 关键词1 | 关键词2 | 关键词3 |
| -------|--------|--------|
| 0      |  0.5   |   0    |



In [48]:
import jieba.analyse
import simplejson as json
import math

top100 = {a[0]: a[1] for a in json.load(open('sorted_keys.json', mode='r', encoding='utf-8'))[:200]}
del top100['总体']
del top100['帮别人']

product_keys = dict(keys)
product_keys['轻薄'] = 1
right = 0
wrong = 0
for comment in marked_col.find():
    tags = jieba.analyse.extract_tags(comment['content'], topK=20, withWeight=True, allowPOS=('ns', 'n', 'nz'))
    predict = None
    
    if len(tags):
        maybe = 0
        for tag in tags:
            if tag[0] in top100 or tag[0] in product_keys:
                maybe += 1
        predict = 0 if maybe else 1
    else:
        predict = 1
    
    if comment.get('commentTags'):
        predict = 0
        
    if predict == comment['mark']:
        right += 1
    else:
        wrong += 1
        print('Marked: ', comment['mark'], 'Predict: ', predict, comment['content'], tags)
    
print('Right:', right, 'Wrong:', wrong)

Marked:  0 Predict:  1 送货很迅速，很轻薄的样子，蛮好看的，可惜电脑包没得送 [('电脑包', 6.25719164545), ('样子', 2.86637914451)]
Marked:  1 Predict:  0 电脑帮朋友下单，应该不错 [('下单', 3.4783139164), ('电脑', 2.4740560032566665), ('朋友', 1.9490361609466669)]
Marked:  1 Predict:  0 太好了了，机子超赞 [('机子', 11.8212361103)]
Marked:  1 Predict:  0 联想在中国是畅销第一的大品牌，，我觉得还可以了，， [('品牌', 3.017744848895), ('中国', 1.51366034333)]
Marked:  1 Predict:  0 很好  不错的电脑套 [('电脑', 7.42216800977)]
Marked:  1 Predict:  0 送货速度很快，我最喜欢送货师傅的态度 [('态度', 2.94698840327), ('速度', 2.782983015075)]
Marked:  1 Predict:  0 忘记评价了还不错 [('评价', 6.74919218807)]
Marked:  1 Predict:  0 不错，电脑用起来挺好的，赞 [('电脑', 7.42216800977)]
Marked:  1 Predict:  0 不错，速度挺快的。 [('速度', 5.56596603015)]
Marked:  0 Predict:  1 宝贝看着很酷，才开始玩，感觉可以！！ [('感觉', 5.64147828931)]
Marked:  1 Predict:  0 给单位买的，办公首选，联想国产 [('办公', 2.661491863123333), ('国产', 2.56069251076), ('单位', 1.7869149911966666)]
Marked:  0 Predict:  1 快递有点满，总的来说还行 [('有点', 5.5984118571)]
Marked:  0 Predict:  1 帮朋友买的，还不错
 [('朋友', 5.84710848284)]
Marked:  0

In [63]:
import jieba.analyse
import json
import math

pos = 0
words = {}
for (k, v) in json.load(open('sorted_keys.json', mode='r', encoding='utf-8'))[:1000]:
    words[k] = (v, pos)
    pos += 1
fraq = []

def add_word(w):
    global pos
    if words.get(w) is not None:
        return words[w]
    return None

def convert_to_matrix():
    point_words = words.keys()
    results = []
    for single in fraq:
        single_result = [0]*(len(point_words) + 2)
        for (index, weight) in single.items():
            single_result[index] = weight
        results.append(single_result)
    return (list(point_words), results)


for comment in comment_col.find()[:50000]:
    single = {}
    for (tag, weight) in jieba.analyse.textrank(comment['content'], topK=20, withWeight=True, allowPOS=('nz')):
        index = add_word(tag)
        if index is not None:
            single[index[1]] = weight * math.log10(index[0])
    #single[-1] = comment['userLevelId']
    #single[-2] = comment['usefulVoteCount']
    fraq.append(single)

point_words, matrix = convert_to_matrix()
print(point_words[:10])
print('Words Count:', len(point_words))

['个性', '火影', '机都', '指纹', '次数', '频率', '市场', '联网', '寝室', '过瘾']
Words Count: 1000


## K-Means 进行聚类

In [67]:
from sklearn.cluster import KMeans
clf = KMeans(n_clusters=3)
s = clf.fit(matrix[:40000])
print(s)
print(clf.cluster_centers_)
print(clf.labels_)
print(clf.labels_[2])
#i = 1
# while i <= len(clf.labels_):
#     print (i, clf.labels_[i-1])
#     i = i + 1
print(clf.inertia_)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)
[[  5.06224122e-03   7.36572965e-03   1.08302836e-01 ...,   1.92107072e-17
    0.00000000e+00   0.00000000e+00]
 [  3.71184774e+00   2.51094648e-01   3.27228196e-01 ...,   6.00016581e-04
    0.00000000e+00   0.00000000e+00]
 [  2.20411983e-01   3.55994921e+00   2.40254866e-01 ...,   2.23616698e-19
    0.00000000e+00   0.00000000e+00]]
[0 0 0 ..., 0 1 0]
0
346670.429865


In [68]:
results = defaultdict(lambda: [])
pos = 0
for comment in comment_col.find()[:40000]:
    results[clf.labels_[pos]].append(comment['content'])
    pos += 1

for (k, v) in results.items():
    print('Key:', k)
    print(v[:5])

Key: 0
['整天感觉不错', '嗯，，，，确实有点卡，可能是win8没用习惯吧！有点失望', '很不错', '买完就降价了，微信购买才3099多掏了两百块钱。本来想买宏基的显卡比较好一点，就是不带DVD所以放弃了。过段时间在发图了。', '挺好能再便宜就好了。']
Key: 1
['总体说电脑不错，屏幕大。。', '电脑不错，只是分区不合理，开合时，会有异想', '电脑不错，速度比较快，好评', '电脑可以，玩游戏很顺畅。支持T5', '很棒！电脑很漂亮！确实有漏光，但是我个人觉得一点也不影响']
Key: 2
['笔记本很漂亮，运行速度也很好，尤其是做工，三星的东西还是有品质啊，很愉快的一次购物，希望质量给力', '刚拿到手，还可以，没太感觉到ssd的速度,稳定就行了.', '质量不错，运行速度很快，显示效果不错', '很喜欢，产品与描述一致，很适合少女，目前使用速度超快，也没有发烫的现象。。。', '开机速度确实很快,非常漂亮,很薄,第一次用window10，习惯就好了,很满意,，，，']


In [36]:
import json
r = {}
sum_fraq = defaultdict(lambda: 0)
for (skuid, contents) in results.items():
    r[skuid] = list(reversed(sorted(contents.items(), key=lambda x: x[1])))
    for (k,c) in contents.items():
        sum_fraq[k] += c
for (skuid, after) in r.items():
    product = product_col.find_one({'skuid': skuid})
    
json.dump(
    list(reversed(sorted(sum_fraq.items(), key=lambda x: x[1]))), 
    open('sorted_keys.json', mode='w', encoding='utf-8'),
    ensure_ascii=False,
    indent=2
)

In [2]:
import json
keys = json.load(open('sorted_keys.json', mode='r', encoding='utf-8'))
key_map = {n[0]: n[1] for n in keys}
key_map

{'军工': 3,
 '防脏': 1,
 '高质量': 14,
 '补水': 1,
 '独卡': 1,
 '份京豆': 1,
 '对方': 25,
 '平衡器': 1,
 '光线': 29,
 '手放': 1,
 '乞丐': 10,
 '代工': 13,
 '记笔记': 2,
 '下鲁': 4,
 '右手边': 12,
 '帮弟': 1,
 '死样': 1,
 '末治者': 1,
 '麻袋': 1,
 '代理': 3,
 '右手掌': 2,
 '水军': 3,
 '小卡': 1,
 '大学': 137,
 '新鲜': 14,
 '真版': 1,
 '货太': 1,
 '瑞都': 1,
 '半面': 1,
 '解决方案': 17,
 '通行证': 1,
 '茨品': 1,
 '位数': 2,
 '时能': 12,
 '线头': 2,
 '会顶': 1,
 '直流': 1,
 '题库': 1,
 '信心': 21,
 '外置': 43,
 '保价': 33,
 '弊端': 2,
 '件产品': 1,
 '热情': 237,
 '标题': 7,
 '货蒸价': 1,
 '精彩': 1,
 '女方': 1,
 '纸壳': 2,
 '业务知识': 1,
 '卡翻': 1,
 '妹夫': 1,
 '全部都是': 4,
 '专玩': 1,
 '待处理': 1,
 '水泡': 1,
 '乱提': 1,
 '产品包装': 4,
 '平民化': 1,
 '个空': 1,
 '额额': 6,
 '走路': 2,
 '红安': 2,
 '颈椎病': 5,
 '边缝': 1,
 '珠峰': 3,
 '预料': 15,
 '理解万岁': 2,
 '平分': 4,
 '妹用': 1,
 '老板': 248,
 '结实': 179,
 '小玩意儿': 1,
 '系统内存': 7,
 '杂声': 4,
 '升学': 1,
 '小黑点': 6,
 '中轻': 3,
 '临时工': 1,
 '角落': 6,
 '堂堂': 1,
 '满嘴': 1,
 '黄油': 1,
 '眼见': 1,
 '错款': 1,
 '先吧': 1,
 '公版': 5,
 '手气': 1,
 '温情': 1,
 '大根': 1,
 '出远门': 1,
 '系统日志': 2,
 '跳帧': 8,
 '热线电话': 2,
 '土鳖'